In [6]:
import pandas as pd
import nbformat
# Load the datasets
combined_age_df = pd.read_csv("data/combined_age.csv")
death_rate_df = pd.read_csv('data/death_rate_corrected.csv')
education_data_df = pd.read_csv('data/education_data.csv')
cdc_diagnosed_df = pd.read_csv('data/cdc_diagnosed_transformed.csv')

# Convert 'Year' to numeric, round to the nearest whole number, and convert to int for all DataFrames
for df in [combined_age_df, death_rate_df, education_data_df, cdc_diagnosed_df]:
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').round().astype('Int64').dropna()

# Merge death rate, education, and CDC diagnosed data on 'Year' and 'State' using outer join
merged_df = pd.merge(death_rate_df, education_data_df, on=['Year', 'State'], how='outer', suffixes=('_death', '_edu'))
merged_df = pd.merge(merged_df, cdc_diagnosed_df, on=['Year', 'State'], how='outer')

# Merge the cleaned and filtered combined_age_df with the previously merged DataFrame using outer join
final_merged_df_with_nan = pd.merge(merged_df, combined_age_df, on=['Year', 'State'], how='outer')

# Reorder columns to have 'Year' and 'State' at the beginning
columns = ['Year', 'State'] + [col for col in final_merged_df_with_nan.columns if col not in ['Year', 'State']]
final_merged_df_with_nan = final_merged_df_with_nan[columns]

# Sort the final merged DataFrame by 'Year' in ascending order
final_merged_df_with_nan = final_merged_df_with_nan.sort_values(by=['Year', 'State'])

# Round specified columns to the nearest tenth of a percent
columns_to_round = [
    "Diagnosed Diabetes (Percentage)", 
    "Obesity (Percentage)",
    "Newly Diagnosed Diabetes (Rate per 1000)",  # Assuming this was meant instead of "(Percentage)"
    "Newly Diagnosed Physical Inactivity (Percentage)"
]

for col in columns_to_round:
    if col in final_merged_df_with_nan.columns:
        final_merged_df_with_nan[col] = final_merged_df_with_nan[col].round(1)

# Save the final merged DataFrame to a CSV file
final_merged_df_with_nan.to_csv('final_merged_all_years.csv', index=False)

# Display the first few rows of the final merged DataFrame
print(final_merged_df_with_nan.head())


     Year       State  Rate Deaths  Percentage Lower Limit  Upper Limit  \
452  2000     Alabama   NaN    NaN         9.7         7.2           13   
453  2000      Alaska   NaN    NaN  Suppressed  Suppressed   Suppressed   
454  2000     Arizona   NaN    NaN  Suppressed  Suppressed   Suppressed   
455  2000    Arkansas   NaN    NaN         8.5         5.8         12.3   
456  2000  California   NaN    NaN        13.3         9.3         18.7   

     Diagnosed Diabetes (Percentage)   Obesity (Percentage)   \
452                               NaN                    NaN   
453                               NaN                    NaN   
454                               NaN                    NaN   
455                               NaN                    NaN   
456                               NaN                    NaN   

     Newly Diagnosed Diabetes (Rate per 1000)  \
452                                       NaN   
453                                       NaN   
454              

In [7]:
# Load the datasets
combined_age_df = pd.read_csv("data/combined_age.csv")
death_rate_df = pd.read_csv('data/death_rate_corrected.csv')
education_data_df = pd.read_csv('data/education_data.csv')
cdc_diagnosed_df = pd.read_csv('data/cdc_diagnosed_transformed.csv')

# Convert 'Year' to numeric, round to the nearest whole number, and convert to int for all DataFrames
for df in [combined_age_df, death_rate_df, education_data_df, cdc_diagnosed_df]:
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').round().astype('Int64').dropna()

# Find common years across all datasets
common_years = set(combined_age_df['Year']).intersection(death_rate_df['Year'], education_data_df['Year'], cdc_diagnosed_df['Year'])

# Filter each dataset to include only rows with common years
combined_age_df = combined_age_df[combined_age_df['Year'].isin(common_years)]
death_rate_df = death_rate_df[death_rate_df['Year'].isin(common_years)]
education_data_df = education_data_df[education_data_df['Year'].isin(common_years)]
cdc_diagnosed_df = cdc_diagnosed_df[cdc_diagnosed_df['Year'].isin(common_years)]

# Merge death rate, education, and CDC diagnosed data on 'Year' and 'State'
merged_df = pd.merge(death_rate_df, education_data_df, on=['Year', 'State'], how='inner', suffixes=('_death', '_edu'))
merged_df = pd.merge(merged_df, cdc_diagnosed_df, on=['Year', 'State'], how='inner')

# Merge the cleaned and filtered combined_age_df with the previously merged DataFrame
final_merged_df = pd.merge(merged_df, combined_age_df, on=['Year', 'State'], how='inner')

# Reorder columns to have 'Year' and 'State' at the beginning
columns = ['Year', 'State'] + [col for col in final_merged_df.columns if col not in ['Year', 'State']]
final_merged_df = final_merged_df[columns]

# Sort the final merged DataFrame by 'Year' in ascending order
final_merged_df = final_merged_df.sort_values(by=['Year', 'State'])

# Round specified columns to the nearest tenth of a percent
columns_to_round = [
    "Diagnosed Diabetes (Percentage)", 
    "Obesity (Percentage)",
    "Newly Diagnosed Diabetes (Rate per 1000)",  # Assuming this was meant instead of "(Percentage)"
    "Newly Diagnosed Physical Inactivity (Percentage)"
]

for col in columns_to_round:
    if col in final_merged_df.columns:
        final_merged_df[col] = final_merged_df[col].round(1)


# Save the final merged DataFrame to a CSV file
final_merged_df.to_csv('final_merged_common_years.csv', index=False)
# Display the first few rows of the final merged DataFrame
print(final_merged_df.head())

     Year       State  Rate Deaths Percentage Lower Limit  Upper Limit  \
98   2014     Alabama  22.8  1,281       14.9        12.6         17.7   
99   2014      Alaska  19.4    113        9.2         6.3         13.3   
100  2014     Arizona  24.3  1,936         14        11.7         16.6   
101  2014    Arkansas  24.0    828       14.6        11.2         18.8   
102  2014  California  20.4  8,249       15.3        13.2         17.8   

     Diagnosed Diabetes (Percentage)   Obesity (Percentage)   \
98                          10.544776              30.029851   
99                           7.962963              26.188889   
100                          8.940000              28.360000   
101                          8.906667              27.588000   
102                          7.841379              23.558621   

     Newly Diagnosed Diabetes (Rate per 1000)  \
98                                       12.3   
99                                        7.3   
100                    